In [4]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import pandas as pd
# Heston call price
def Heston_call_price(S0, v0, K, T, r, q, kappa, theta, sigma, rho, lmbda):
    p1 = p_Heston(S0, v0, K, r, q, T, kappa, theta, sigma, rho, lmbda, 1)
    p2 = p_Heston(S0, v0, K, r, q, T, kappa, theta, sigma, rho, lmbda, 2)
    return S0 * np.exp(-q*T) * p1 - K * np.exp(-r*T) * p2

# Heston probability
def p_Heston(S0, v0, K, r, q, T, kappa, theta, sigma, rho, lmbda, j):
    integrand = lambda phi: np.real(np.exp(-1j * phi * np.log(K)) \
                                    * f_Heston(phi, S0, v0, T, r, q, kappa, theta, sigma, rho, lmbda, j) \
                                    / (1j * phi))    
    integral = quad(integrand, 0, 100)[0]
    return 0.5 + (1 / np.pi) * integral

# Heston characteristic function
def f_Heston(phi, S0, v0, T, r, q, kappa, theta, sigma, rho, lmbda, j):
        
    if j == 1:
        u = 0.5
        b = kappa + lmbda - rho * sigma
    else:
        u = -0.5
        b = kappa + lmbda
    
    a = kappa * theta
    d = np.sqrt((rho * sigma * phi * 1j - b)**2 - sigma**2 * (2 * u * phi * 1j - phi**2))
    g = (b - rho * sigma * phi * 1j + d) / (b - rho * sigma * phi * 1j - d)
    C = (r - q) * phi * 1j * T + (a / sigma**2) \
            * ((b - rho * sigma * phi * 1j + d) * T - 2 * np.log((1 - g * np.exp(d * T))/(1 - g)))
    D = (b - rho * sigma * phi * 1j + d) / sigma**2 * ((1 - np.exp(d * T)) / (1 - g * np.exp(d * T)))
    
    return np.exp(C + D * v0 + 1j * phi * np.log(S0))

In [2]:
# Parameters
T = 1        # maturity
S0 = 55      # spot price
K = 50       # strike price
r = 0.04     # risk-free interest rate
q = 0.02     # dividend rate
v0 = 0.04    # initial variance
rho = -0.7   # correlation between Brownian motions
kappa = 2    # mean reversion rate
theta = 0.04 # Long term mean of variance
sigma = 0.3  # volatility of volatility
lmbda = 0    # market price of volatility risk

# Option values
Vc = Heston_call_price(S0, v0, K, T, r, q, kappa, theta, sigma, rho, lmbda) # call
Vp = Vc + K*np.exp(-r*T) - S0*np.exp(-q*T) # put with put-call parity

print('Call price: ' + str(round(Vc, 5)))
print('Put price:  ' + str(round(Vp, 5)))

Call price: 7.78614
Put price:  1.91469


In [5]:
df = pd.read_csv('E:\RoboAdvisor\merged_test.csv')
df = df.dropna(subset=['stockPrice'])
df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)
df['Expiration_Date'] = pd.to_datetime(df['Expiration_Date'])
df['Time_to_Expiration'] = (
    df['Expiration_Date'] - df['Datetime']) / pd.Timedelta(days=365)
df = df[df['CALL_PUT'] == 'CALL']
df = df[df['contractSymbol'] == 'SPY230915C00455000']

In [ ]:
option_prices = []
for index, row in df.iterrows():
    S = row['stockPrice']
    K = row['Strike']
    T = row['Time_to_Expiration']
    r = 0.04     # risk-free interest rate
    q = 0.02     # dividend rate
    v0 = 0.1    # initial variance
    rho = -0.5   # correlation between Brownian motions
    kappa = 2    # mean reversion rate
    theta = 0.1 # Long term mean of variance
    sigma = 0.3  # volatility of volatility
    lmbda = 0    # market price of volatility risk
    option_price = Heston_call_price(S0, v0, K, T, r, q, kappa, theta, sigma, rho, lmbda)
    option_prices.append(option_price)
df['Option_Price'] = option_prices